<center><h3>Universidade Federal de Pelotas - Cursos de Ciência e Engenharia de Computação</h3></center> 
<center><h4>Disciplina: Cálculo Numérico Computacional Profa. Larissa A. de Freitas</h4> </center> 


<center><h2><strong>Relatório 1 – Resolução Numérica de Equações Algébricas e Transcendentes<strong></h2> </center> 

Questão 1: A função f(x) = tg(πx) - 6 tem um zero em (1/π) arctg(6) ≈ 0.447431543. Sejam x0 = 0 e x1 = 0.48. Utilize 10 iterações de cada um dos métodos a seguir para encontrar a aproximação dessa raiz. Qual é o método mais eficiente e por quê?

a. Método da Bissecção

b. Método da Falsa Posição
    
c. Método da Secante

Faça a comparação dos erros relativos percentuais para os métodos para determinar a raiz da função.


Questão 2: O polinômio de quarto grau f (x) = 230x4 + 18x3 + 9x2- 221x - 9, tem dois zeros reais, um em [-1, 0] e outro em [0, 1]. Tente encontrar a aproximação desses zeros com a precisão de 10-6, utilizando:

a. Método da Falsa Posição

b. Método da Secante

c. Método de Newton-Raphson/Tangente

d. Plote um gráfico para explicar a diferença entre os resultados obtidos pelos Métodos da Falsa Posição e da Secante 

Utilize as extremidades de cada intervalo como aproximações iniciais em (a) e
(b) e os pontos médios como as aproximações iniciais em (c).

Questão 3: Uma gamela de comprimento L tem seção transversal semicircular com raio r (veja a figura abaixo). Quando a gamela está cheia com água até uma distância h do topo, o volume V de água é V = L [0,5πr²- r²arcsen(h/r) - h(r² – h²)½]

![alt text](imagem_relatorio.PNG)

Suponha que L = 10 pés, r = 1 pé e V = 12,4 pés. Usando o Método da Bissecção, determine a profundidade da água na gamela com precisão de 0,01 pé.

Questão 4: Você está projetando um tanque esférico (veja a figura abaixo) para armazenar água para uma pequena cidade num país

![alt text](imagem_relatorio2.PNG)

O volume de líquido que ele pode armazenar pode ser calculado por V =π h² [3 R−h]/3

Onde V é o volume (m³), h é a profundidade de água no tanque (m) e R é o raio do tanque (m). Se R = 3m, até qual profundidade o tanque deve ser enchido para conter 30 m³? Use 3 iterações do Método do Newton-Raphson/Tangente para obter a resposta. 


Questão 5: O Método de Newton-Raphson/Tangente pode não ser convergente. Verifique que, para as funções e estimativas iniciais mostradas em (a)e (b), o método não funciona.

a. f(x) = 1/10 + xe⁻ˣ com x₁ = 1

b. f(x) = x³ - x - 3  com x₁ = -3

### **Imports**

In [72]:
from math import e as euler
from math import tan as tan
from math import pi as pi
from math import asin as asin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

## **Tabelas**

In [73]:
REPETITIONS   = 'n\u207F'
LEFT_BOUND    = 'a\u207F'
RIGHT_BOUND   = 'b\u207F'
MIDDLE_POINT  = 'x\u207F'
FX            = 'f(x\u207F)'
FA            = 'f(a\u207F)'
FA_FX         = 'f(a\u207F) * f(x\u207F)'

table_values_bisection = {
    REPETITIONS: [],
    LEFT_BOUND: [],
    RIGHT_BOUND: [],
    MIDDLE_POINT: [],
    FX: [],
    FA: [],
    FA_FX: []
}

table_values_false_position = {
    REPETITIONS: [],
    LEFT_BOUND: [],
    RIGHT_BOUND: [],
    MIDDLE_POINT: [],
    FX: []
}

table_values_newton = {
    REPETITIONS: [],
    MIDDLE_POINT: [],
    FX: []
}

## **Funções auxiliares**


In [74]:
def plot_function( a, b, func, root ):
    x_values = np.linspace( -2, 2, 1000 )
    plt.rcParams['figure.figsize'] = [5, 5]
    plt.rcParams['figure.facecolor'] = '#282a36'
    plt.rcParams['figure.autolayout'] = True

    plt.tick_params( axis='x', colors='white' )
    plt.tick_params( axis='y', colors='white' )

    plt.scatter( a, func( a ), color='green', label='Bound Left' )
    plt.scatter( b, func( b ), color='green', label='Bound Right' )
    plt.scatter( root, func( root ), color='red', marker='x', label='Root' )
    plt.grid( True )
    plt.plot( x_values, np.vectorize(func)(x_values), label='f(x) = x^3 - 9x + 5' )
    plt.show()

def average( a, b ):
    return ( a + b ) / 2

def weighted_average( a, b, f_a, f_b ):
    return (  a * f_b  -  b * f_a )  / ( f_b - f_a )

def product( x1, x2 ):
    return x1 * x2 

def display_bissection_table():
    df = pd.DataFrame( table_values_bisection )
    df.columns = [REPETITIONS, LEFT_BOUND, RIGHT_BOUND, MIDDLE_POINT, FX, FA, FA_FX]
    display( df.style.hide( axis="index" ).format( na_rep = None, precision = 4 ) )
    clear_table(table_values_bisection)

def display_false_position_table():
    df = pd.DataFrame( table_values_false_position )
    df.columns = [REPETITIONS, LEFT_BOUND, RIGHT_BOUND, MIDDLE_POINT, FX]
    display( df.style.hide( axis="index" ).format( na_rep = None, precision = 6 ) )
    clear_table(table_values_false_position)

def add_values( repetitions= None , a= None , b= None , x= None , f_x= None , f_a= None , f_x_f_a= None  ):
    table_values_bisection[REPETITIONS].append( repetitions )
    table_values_bisection[LEFT_BOUND].append( a )
    table_values_bisection[RIGHT_BOUND].append( b )
    table_values_bisection[MIDDLE_POINT].append( x )
    table_values_bisection[FX].append( f_x )
    table_values_bisection[FA].append( f_a )
    if f_x_f_a > 0:
        table_values_bisection[FA_FX].append( '> 0' )
    elif f_x_f_a < 0:
        table_values_bisection[FA_FX].append( '< 0' )

def add_false_position_values( repetitions= None , a= None , b= None , x= None , f_x= None ):
    table_values_false_position[REPETITIONS].append( repetitions )
    table_values_false_position[LEFT_BOUND].append( a )
    table_values_false_position[RIGHT_BOUND].append( b )
    table_values_false_position[MIDDLE_POINT].append( x )
    table_values_false_position[FX].append( f_x )
    print

def clear_table(table):
    for key in table:
        table[key] = []

green_color = "\033[32m"  # Verde
blue_color = "\033[34m"   # Azul
red_color = "\033[31m"    # Vermelho
reset_color = "\033[0m"   # Resetar para a cor padrão

## **Método da Bissecção**

<li>Se function(left) x function(middle) > 0   então raiz estao no intervalo [ middle, right ], logo left = middle </li>
<li>Senão se function(left) . function(middle) < 0    então raiz estao no intervalo [ left, middle ], logo right = middle </li>

### **Parametros**

<li>left: Limite a esquerda do intervalo.</li>
<li>right: Limite a direita do intervalo.</li>

### **Critérios de paradas**
<li>e: Epsilon;</li>

<li>n: Número de iterações.</li>

**Return middle: Raiz aproximada.**


In [75]:
def bisection( left , right , e = None, n = None, func = None ):
    counter = 0
    if ( e is not None and n is not None ) or ( e is None and n is None ):
        raise ValueError( 'You must provide either the number of iterations or the epsilon margin' )
    if func is None:
        raise ValueError('You must provide a function')
    #CASO 1 ONDE SO E DADA A PRECISAO
    if e is not None and n is None:
        while abs( right - left )  >= e:
            counter += 1
            middle = average( left, right )
            prod = product( func( left ), func( middle ) )
            add_values( counter, left, right, middle, func( middle ), func( left ), prod )
            if prod > e:
                left = middle
            else:
                if prod < e:
                    right = middle
    #CASO 2 ONDE SO E DADO O NUMERO DE ITERACOES
    elif e is None and n is not None:                       
        for iteration in range(1, n):
            counter += 1
            middle = average( left, right )
            prod = product( func( left ), func( middle ) )
            add_values( counter, left, right, middle, func( middle ), func( left ), prod )
            if prod > 0:
                left = middle
            else:
                if prod < 0:
                    right = middle	
    #CASO 3 ONDE E DADO O NUMERO DE ITERACOES E A PRECISAO
    elif e is not None and n is None:
        while abs( right - left )  >= e:
            if counter == n:
                break
            counter += 1
            middle = average( left, right )
            prod = product( func( left ), func( middle ) )
            add_values( counter, left, right, middle, func( middle ), func( left ), prod )
            if prod > e:
                left = middle
            else:
                if prod < e:
                    right = middle
    return middle

In [76]:
def false_positon( left, right, e = None, n = None , func = None ):
    counter = 0
    if ( e is not None and n is not None ) or ( e is None and n is None ):
        raise ValueError( 'You must provide either the number of iterations or the epsilon margin' )
    
    if func(left) * func(right) >= 0:
       raise ValueError( 'The function does not change sign in the interval' )
    
    #CASO 1 ONDE SO E DADA A PRECISAO
    if e is not None and n is None:
        while abs( right - left ) >= e:
            counter += 1
            middle = weighted_average( left, right, func( left ), func( right ) )
            if abs(func( middle )) < e:
                return middle
            prod = product( func( left ), func( middle ) )
            add_false_position_values( counter, left, right, middle, func( middle ))
            if prod > e:
                left = middle
            else:
                if prod < e:
                    right = middle
    #CASO 2 ONDE SO E DADO O NUMERO DE ITERACOES
    elif n is not None and e is None:
        for iteration in range(1, n):
            counter += 1
            middle = weighted_average( left, right, func( left ), func( right ) )
            prod = product( func( left ), func( middle ) )
            add_false_position_values( counter, left, right, middle, func( middle ))
            if prod > 0:
                left = middle
            else:
                if prod < 0:
                    right = middle          
    #CASO 3 ONDE E DADO O NUMERO DE ITERACOES E A PRECISAO
    elif n is not None and e is not None:
        while ( abs( right - left ) ) >= e:
            if counter == n:
                break
            counter += 1
            middle = weighted_average( left, right, func( left ), func( right ) )
            if abs(func( middle )) < e:
                return middle
            prod = product( func( left ), func( middle ) )
            add_false_position_values( counter, left, right, middle, func( middle ))
            if prod > e:
                left = middle
            else:
                if prod < e:
                    right = middle
    return middle
       

In [77]:
def newton_raphson( x0, n = None, e = None, func = None, dfunc = None ):
    if e is None and n is None:
        raise ValueError('You must provide either the number of iterations or the epsilon margin')
    if func is None or dfunc is None:
        raise ValueError('You must provide a function')
    #CASO 1 ONDE SO E DADA A PRECISAO
    if e is not None and n is None:
        while abs(func(x0)) >= e:
            if dfunc(x0) == 0:
                return x0
            i = x0 - (func(x0)/dfunc(x0))
            if abs(func(i)) < e:
                return i
            else:
                if abs(i - x0) < e:
                    return i
            x0 = i
        pass
    #CASO 1 ONDE SO E DADA A PRECISAO
    elif e is None and n is not None:
        for iteration in range(1, n):
            if abs(func(x0)) < e:
                return x0
            if dfunc(x0) == 0:
                return x0
            i = x0 - (func(x0)/dfunc(x0))
            if abs(func(i)) < e:
                return i
            else:
                if abs(i - x0) < e:
                    return i
            x0 = i
    #CASO 3 ONDE E DADO O NUMERO DE ITERACOES E A PRECISAO
    elif e is not None and n is not None:
        for iteration in range(1, n):
            if abs(func(x0)) < e:
                return x0
            if dfunc(x0) == 0:
                return x0
            i = x0 - (func(x0)/dfunc(x0))
            if abs(func(i)) < e:
                return i
            else:
                if abs(i - x0) < e:
                    return i
            x0 = i
    return x0

In [78]:
def secant(x0, x1, e,  n, func):
    if e is None and n is None:
        raise ValueError('You must provide either the number of iterations or the epsilon margin')
    if func is None:
        raise ValueError('You must provide a function')
    
    #CASO 1 ONDE SO E DADA A PRECISAO
    if e is not None and n is None:
        while abs(func(x1)) >= e or abs(x1 - x0) >= e:
            x2 = x1 - (func(x1) * (x1 - x0) / (func(x1) - func(x0)))
            x0 = x1
            x1 = x2
    
    #CASO 2 ONDE SO E DADO O NUMERO DE ITERACOES
    elif e is None and n is not None:
        for iteration in range(1, n):
            x2 = x1 - (func(x1) * (x1 - x0) / (func(x1) - func(x0)))
            x0 = x1
            x1 = x2
            
    #CASO 3 ONDE E DADO O NUMERO DE ITERACOES E A PRECISAO
    elif e is not None and n is not None:
        for iteration in range(1, n):
            if abs(func(x0)) < e:
                return x0
            if abs(func(x1)) < e or abs(x1 - x0) < e:
                return x1
            
            x2 = x1 - (func(x1) * (x1 - x0) / (func(x1) - func(x0)))
            if abs(func(x2)) < e or abs(x2 - x1) < e:
                return x2
            x0 = x1
            x1 = x2
    
    return x1


### **Questão 1**

In [79]:

try:
    root_bisection      = bisection     ( 0, 0.48, None, 10, lambda x: tan( pi*x ) - 6 )
    root_false_position = false_positon ( 0, 0.48, None, 10, lambda x: tan( pi*x ) - 6 )
    root_sec            = secant        ( 0, 0.48, None, 10, lambda x: tan( pi*x ) - 6 )
    print(f"Raiz encontrada pelo método da bisseção: {green_color}{root_bisection}{reset_color}")
    print(f"Raiz encontrada pelo método da falsa posição: {green_color}{root_false_position}{reset_color}")
    print(f"Raiz encontrada pelo método da secante: {green_color}{root_sec}{reset_color}")
    display_false_position_table()
    display_bissection_table()

except ValueError as e:
    print( e )



2
Raiz encontrada pelo método da bisseção: 0.44718749999999996
Raiz encontrada pelo método da falsa posição: 0.44206694908170113
Raiz encontrada pelo método da secante: -195.89499482451663


nⁿ,aⁿ,bⁿ,xⁿ,f(xⁿ)
1,0.000000,0.480000,0.181194,-5.360105
2,0.181194,0.480000,0.286187,-4.742211
3,0.286187,0.480000,0.348981,-4.052821
4,0.348981,0.480000,0.387053,-3.301069
5,0.387053,0.480000,0.410305,-2.545638
6,0.410305,0.480000,0.424566,-1.859550
7,0.424566,0.480000,0.433336,-1.295153
8,0.433336,0.480000,0.438737,-0.868485
9,0.438737,0.480000,0.442067,-0.566358


nⁿ,aⁿ,bⁿ,xⁿ,f(xⁿ),f(aⁿ),f(aⁿ) * f(xⁿ)
1,0.0000,0.4800,0.2400,-5.0609,-6.0000,> 0
2,0.2400,0.4800,0.3600,-3.8749,-5.0609,> 0
3,0.3600,0.4800,0.4200,-2.1053,-3.8749,> 0
4,0.4200,0.4800,0.4500,0.3138,-2.1053,< 0
5,0.4200,0.4500,0.4350,-1.1712,-2.1053,> 0
6,0.4350,0.4500,0.4425,-0.5245,-1.1712,> 0
7,0.4425,0.4500,0.4462,-0.1343,-0.5245,> 0
8,0.4462,0.4500,0.4481,0.0817,-0.1343,< 0
9,0.4462,0.4481,0.4472,-0.0282,-0.1343,> 0


### **Questão 2**

In [80]:
try:
    #INTERVALO [-1,0]
    root_false_position = false_positon( -1, 0, 10**-6, None, lambda x: 230*x**4 + 18*x**3 + 9*x**2 - 221*x - 9 )
    root_sec            = secant( -1, 0, 10**-6, None, lambda x: 230*x**4 + 18*x**3 + 9*x**2 - 221*x - 9 )
    root_newton         = newton_raphson( -1, 10, 10**-6, lambda x: 230*x**4 + 18*x**3 + 9*x**2 - 221*x - 9, lambda x: 920*x**3 + 54*x**2 + 18*x - 221)
    print("INTERVALO [-1,0]")
    print(f"Raiz encontrada pelo método da falsa posição: {green_color}{root_false_position}{reset_color}")
    print(f"Raiz encontrada pelo método da secante: {green_color}{root_sec}{reset_color}")
    print(f"Raiz encontrada pelo método de Newton-Raphson: {green_color}{root_newton}{reset_color}\n")
except ValueError as e:
    print(e)

root_sec = None
root_newton = None
root_false_position = None
try:
    print("INTERVALO [0,1]")
    #NAO CONSEGUE ACHAR A RAIZ -> root_false_position = false_positon( 0, 1, 10**-6, None, lambda x: 230*x**4 - 18*x**3 + 9*x**2 - 221*x - 9 )
    root_sec            = secant( 0, 1, 10**-6, None, lambda x: 230*x**4 + 18*x**3 + 9*x**2 - 221*x - 9 )
    root_newton         = newton_raphson( 1, 10, 10**-6, lambda x: 230*x**4 + 18*x**3 + 9*x**2 - 221*x - 9, lambda x: 920*x**3 - 54*x**2 + 18*x - 221)
    print(f"Raiz encontrada pelo método da falsa posição: {red_color} NAO CONSEGUE ACHAR A RAIZ{reset_color}")
    print(f"Raiz encontrada pelo método da secante: {green_color}{root_sec}{reset_color}")
    print(f"Raiz encontrada pelo método de Newton-Raphson: {green_color}{root_newton}{reset_color}\n")
    #print(root_newton)

except ValueError as e:
    print(e)

1
INTERVALO [-1,0]
Raiz encontrada pelo método da falsa posição: -0.040659284770926674
Raiz encontrada pelo método da secante: -0.040659288315725135
Raiz encontrada pelo método de Newton-Raphson: -0.04065928831592832

INTERVALO [0,1]
1
Raiz encontrada pelo método da falsa posição:  NAO CONSEGUE ACHAR A RAIZ
Raiz encontrada pelo método da secante: -0.04065928831557162
Raiz encontrada pelo método de Newton-Raphson: 0.9623983200746065



### **Questão 3**

In [81]:
try:
    r = 1
    v = 12.4
    l = 10
    root = bisection(0, 0.3048, 0.01, None, lambda h: l * (0.5*pi*r**2 - r**2*asin(h/r) - h*(r**2 -h**2)**1/2) - v)
    print(root)
    print(l * (0.5*pi*r**2 - r**2*asin(root/r) - root*(r**2 -root**2)**1/2))
except ValueError as e:
    print( e )


0.21907500000000002
12.456496138330994


### **Questão 4**

In [82]:
print("aqui")

aqui


### **Questão 5**

In [83]:
try:
    print("QUESTAO A")
    root_a = newton_raphson (2, 10, 0.0001, lambda x: 1/10 + x*euler**-x, lambda x: -euler**-x*x + euler**-x)
    print(f"Raiz encontrada: {green_color}{root_a}{reset_color}\n")

except ValueError as e:
    print( e )

try:
    print("QUESTAO B")
    root_b = newton_raphson (-3, 10,0.0001, lambda x: x**3 - x - 3, lambda x: 3*x**2 - 1)
    print(f"Raiz encontrada: {green_color}{root_b}{reset_color}\n")

except ValueError as e:
    print( e )

     


QUESTAO A
Raiz encontrada: 7.393684957031746e+47

QUESTAO B
Raiz encontrada: -1.9618786463602413

